In [ ]:
import numpy as np

import glue

In [ ]:
x = np.random.randn(100)
y = x*1.2 + np.random.randn(len(x))
z = x*2.3 + y*3.1 + np.random.randn(len(x))/3.5

w = 1/x + np.random.randn(len(x))/1.1

In [ ]:
# MAYBE:
nbglue = glue.nbglue()

# or perhaps it needs to be singleton-y? if so, then
from glue import nbglue

In [ ]:
nbglue.add(glue.core.Data(x=x, y=y, z=z))

# or have nbglue include Data for convenience:
nbglue.add(nbglue.core.Data(x=x, y=y, z=z))

# or this could be a convenience function... 
# depending on how flexible Data should be (i.e. the below would 
# accept various random things like pandas, astropy tables, etc a
# la qglue):
nbglue.add_dataset({'x':x, 'y':y, 'z':z}, label='dataset')

# In a "blocking" mode (using Qt)

In [ ]:
plot1 = nbglue.scatter('x', 'y')
plot2 = nbglue.scatter('x', 'z')
plot3 = nbglue.scatter_3d('x','y','z')

nbglue.show([plot1, plot2, plot3])

Running the cell above should then pop up three windows, each with their associated plot options.  When all those windows are closed, the function returns jupyter notebook images of the three windows in their final state when closed.  *Maybe* also any changed plot options?

As a slightly-less optimal choice, it could be a "stripped-down" ginga viewer that has all three plots as panes in the standard viewer but with a bunch of the stuff like the plot layers and dataset tree hidden.

Presumably it wouldn't be too hard to also let the plot options get set in the `scatter`/`scatter_3d` calls above.

### Probably something like this should also be possible: 

In [ ]:
dataset2 = glue.core.Data({'x':x, 'w': w}, label='dataset2')
nbglue.add(dataset2)

# the dataset with label '' is the "default" without a specific dataset
nbglue.link('x', 'dataset2.x')

plot4 = nbglue.scatter('dataset2.x', 'dataset2.w')

nbglue.show([plot1, plot2, plot3, plot4])

# In a non-blocking mode (using Qt)

In [ ]:
nbglue.ishow([plot1, plot2, plot3])

Just like the above, but it returns some sort of "delayed image" thingie.  Then once you've played around with the plots, when you save the notebook, it grabs a screenshot and drops it into the "delayed image".  So the last state gets saved.  If you close the window, that's frozen as the "delayed image".

In [ ]:
# might also be possible, at least in the "singleton" mode:
nbglue.finalize()
# this would close all the windows and freeze the "delated image"

### This is a more complex sort of dataset, but illustrates *why* you'd want to do this in an interactive mode

In [ ]:
from photutils import DAOStarFinder
from astropy.stats import mad_std

... load some image into `image`...

bkg_sigma = mad_std(image)    
daofind = DAOStarFinder(fwhm=4., threshold=3.*bkg_sigma)    
sources = daofind(image) 
# The result called `sources` is an astropy table that has
# 'xcentroid', 'ycentroid', and 'mag' columns

In [ ]:
nbglue.add_dataset(sources)

plot1 = nbglue.scatter('xcentroid', 'ycentroid')
plot2 = nbglue.hist('mag')

nbglue.show([plot1, plot2])

Now I go and draw some regions using the glue plotting tools using *both* plots (e.g., blocking out certain ranges in the histogram *and* drawing a circle in the x/y plot).

In [ ]:
msk = nbglue.get_selection('')  
# I'm not sure exactly what the scope of selections is in glue.  So this could also be:
msk = nbglue.get_selection('xcentroid')
# or
msk = plot1.get_selection('xcentroid')
# or even
msk = nbglue.get_selection(sources)

In [ ]:
np.max(sources[msk]['peak'])

Oops!  After doing that I now realize that some of my sources are cosmic rays or something.  So I realize I want to add more.  Two possible ways that could work, one better (but I suspect might be harder to implement), and another less optimal but still ok:

#### Best option 

In [ ]:
nbglue.add_selection(sources['peak']<10000)

Now I examine the plots and I'm happy with my selection

In [ ]:
msk = nbglue.get_selection('') # or whatever above

#### Backup option 

In [ ]:
msk1 = sources['peak']<10000

I just accept that I can't programatically mess with the selection, but am pretty sure that's the right cut

In [ ]:
msk = msk & msk1

####  (continuing from the end of the two options)

In [ ]:
good_targets = sources[msk]

And then I go on with my science workflow and maybe do the same in another band or something

#  Stretch goal: In the notebook 

This is probably a bigger challenge technically.  The idea would be a Python interface similar/identical to the non-blocking mode, but with the plots and interaction tools *in the notebook*  So you can do the linked brushing and such in the notebook.